In [5]:
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load the pre-trained VGG19 model:
vgg19 = VGG19(include_top=False, # sem os layers densos finais de classificacao
              weights='imagenet', # load params treinados nos exemplos do imagenet
              input_shape=(224,224,3),
              pooling=None)

# Freeze all the layers in the base VGGNet19 model:
for layer in vgg19.layers:
  layer.trainable = False # os params nos layers nao vao ser updated no treino

In [6]:
# Instantiate the sequential model and add the VGG19 model:
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model:
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

# Compile the model for training:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])


In [13]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [14]:
batch_size=32

# Define the train and validation data generators:
train_generator = train_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

# randomly rotates images within a 30-degree range
# randomly flip the images horizontally
# rescale the data to between 0 and 1 (by multiplying by 1/255)
# and load the image data into arrays in the “channels last” format.

valid_generator = valid_datagen.flow_from_directory(
    directory='./hot-dog-not-hot-dog/test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42) # only rescales and load the images

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [9]:
model.fit_generator(train_generator,
                    steps_per_epoch=15,
                    epochs=16,
                    validation_data=valid_generator,
                    validation_steps=15)

AttributeError: 'Sequential' object has no attribute 'fit_generator'